**Создание ML-пайплайна, Git и FastAPI**

In [65]:
!pip install xgboost scikit-learn joblib pandas numpy fastapi uvicorn pyngrok

In [66]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [69]:
import pandas as pd
from sklearn.model_selection import train_test_split
file_path = '/content/drive/MyDrive/Laptop_price.csv'
df = pd.read_csv(file_path)
X = df.drop(columns=['Price'])
y = df['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [70]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
import joblib

num_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = X.select_dtypes(include=['object']).columns.tolist()

num_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])


cat_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])


preprocessor = ColumnTransformer([
    ('num', num_transformer, num_features),
    ('cat', cat_transformer, cat_features)
])


pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5))
])

pipeline.fit(X_train, y_train)

joblib.dump(pipeline, '/content/drive/MyDrive/laptop_price_model.pkl')

['/content/drive/MyDrive/laptop_price_model.pkl']

In [71]:
!git init

Reinitialized existing Git repository in /content/.git/


In [72]:
!git add .
!git commit -m "Добавлен ML-пайплайн"

[main 31ff2c4] Добавлен ML-пайплайн
 3 files changed, 75 insertions(+), 1 deletion(-)
 create mode 100644 app.py
 rewrite "drive/MyDrive/Colab Notebooks/\320\224\321\203\320\275\320\270\320\275_Lab1.ipynb" (90%)
 create mode 100644 fastapi.log


In [73]:
!git config --global user.email "morrainyt@gmail.com"
!git config --global user.name "Dunin2302"

In [15]:
!git add .
!git commit -m "Добавлен ML-пайплайн"

[master (root-commit) 958757f] Добавлен ML-пайплайн
 26 files changed, 56188 insertions(+)
 create mode 100644 .config/.last_opt_in_prompt.yaml
 create mode 100644 .config/.last_survey_prompt.yaml
 create mode 100644 .config/.last_update_check.json
 create mode 100644 .config/active_config
 create mode 100644 .config/config_sentinel
 create mode 100644 .config/configurations/config_default
 create mode 100644 .config/default_configs.db
 create mode 100644 .config/gce
 create mode 100644 .config/hidden_gcloud_config_universe_descriptor_data_cache_configs.db
 create mode 100644 .config/logs/2025.05.09/13.40.57.224020.log
 create mode 100644 .config/logs/2025.05.09/13.41.18.092482.log
 create mode 100644 .config/logs/2025.05.09/13.41.26.495445.log
 create mode 100644 .config/logs/2025.05.09/13.41.27.718573.log
 create mode 100644 .config/logs/2025.05.09/13.41.36.263788.log
 create mode 100644 .config/logs/2025.05.09/13.41.36.930373.log
 create mode 100644 drive/MyDrive/Colab Notebooks/111

In [74]:
!git remote add origin https://github.com/Dunin2302/Labagit

error: remote origin already exists.


In [ ]:
!ssh-keygen -t ed25519 -C "morrainyt@gmail.com"

Generating public/private ed25519 key pair.
Enter file in which to save the key (/root/.ssh/id_ed25519): 
/root/.ssh/id_ed25519 already exists.
Overwrite (y/n)? 

In [41]:
!cat ~/.ssh/id_ed25519.pub

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIE9oCjfohN2baT7f2U2PZuLVc19iQa20qTLkNG/0A2uG morrainyt@gmail.com


In [43]:
!ssh-keyscan github.com >> /root/.ssh/known_hosts

# github.com:22 SSH-2.0-0264bb16
# github.com:22 SSH-2.0-0264bb16
# github.com:22 SSH-2.0-0264bb16
# github.com:22 SSH-2.0-0264bb16
# github.com:22 SSH-2.0-0264bb16


In [44]:
!ssh -T git@github.com

Hi Dunin2302! You've successfully authenticated, but GitHub does not provide shell access.


In [45]:
!git remote -v

origin	https://github.com/Dunin2302/Labagit (fetch)
origin	https://github.com/Dunin2302/Labagit (push)


In [46]:
!git add .
!git commit -m "Добавлен ML-пайплайн"

[master fa9278c] Добавлен ML-пайплайн
 3 files changed, 9 insertions(+), 1 deletion(-)
 rewrite "drive/MyDrive/Colab Notebooks/\320\224\321\203\320\275\320\270\320\275_Lab1.ipynb" (88%)
 create mode 100644 key
 create mode 100644 key.pub


In [47]:
!git config --global credential.helper wincred

In [55]:
!git remote set-url origin git@github.com:Dunin2302/Labagit.git

In [59]:
!git remote add origin git@github.com:Dunin2302/Labagit.git
!git branch -M main
!git push -u origin main

error: remote origin already exists.
Branch 'main' set up to track remote branch 'main' from 'origin'.
Everything up-to-date


**Развертывание **FastAPI**

In [60]:
%%writefile app.py
from fastapi import FastAPI, File, UploadFile
import pandas as pd
import joblib
from io import BytesIO

app = FastAPI()

model_path = "/content/drive/MyDrive/laptop_price_model.pkl"
model = joblib.load(model_path)

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
content = await file.read()

df = pd.read_csv(BytesIO(content))

predictions = model.predict(df)

return {"predictions": predictions.tolist()}

Writing app.py


In [61]:
!pip install python-multipart

In [62]:
!ngrok config add-authtoken 2x0L4CxzTB4twUdO5eh8ghW8vKx_4twuc8bixA6YYvmERb2Ux

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [63]:
!nohup uvicorn app:app --host 0.0.0.0 --port 8000 --reload > fastapi.log 2>&1 &

In [64]:
from pyngrok import ngrok
public_url = ngrok.connect(8000)
print("API доступно по адресу:", public_url)

API доступно по адресу: NgrokTunnel: "https://bd54-34-91-5-2.ngrok-free.app" -> "http://localhost:8000"
